In [1]:
!wget https://www.spataru.at/students/course_files/week4/loan.csv

--2024-11-26 11:34:29--  https://www.spataru.at/students/course_files/week4/loan.csv
Resolving www.spataru.at (www.spataru.at)... 81.19.159.92
Connecting to www.spataru.at (www.spataru.at)|81.19.159.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38013 (37K) [text/csv]
Saving to: ‘loan.csv’

loan.csv            100%[===================>]  37.12K  --.-KB/s    in 0.1s    

2024-11-26 11:34:30 (260 KB/s) - ‘loan.csv’ saved [38013/38013]



In [98]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from  sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import model_selection
import plotly.express as px

In [99]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [100]:
df = pd.read_csv("loan.csv")

In [101]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


# EDA

In [102]:
df.shape

(614, 13)

In [103]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [104]:
df = df.drop_duplicates(subset="Loan_ID")

In [105]:
df.shape

(614, 13)

In [106]:
df.isna().sum()

,0
Loan_ID,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14


In [107]:
imp = SimpleImputer(strategy="most_frequent")

In [108]:
df[df.columns] = imp.fit_transform(df)

In [109]:
df = df.drop(labels=["Loan_ID","Gender","Married", "Education"], axis=1)

In [110]:
df.Dependents.value_counts()

,count
Dependents,
0,360
1,102
2,101
3+,51


In [111]:
df.Dependents.astype("category").values

['0', '1', '0', '0', '0', ..., '0', '3+', '1', '2', '0']
Length: 614
Categories (4, object): ['0', '1', '2', '3+']

In [112]:
df.head()

,Dependents,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,No,5849,0.0,120.0,360.0,1.0,Urban,Y
1,1,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,0,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,0,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,0,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [113]:
category = ["Dependents", "Self_Employed","Credit_History", "Property_Area", "Loan_Status"]

In [114]:
for c in category:
  df[c] = df[c].astype("category")

In [115]:
px.histogram(df, x="ApplicantIncome", color="Loan_Status")

In [116]:
px.scatter(df, x="ApplicantIncome", y="LoanAmount", color="Loan_Status")

In [117]:
df.columns

Index(['Dependents', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status'],
      dtype='object')

In [118]:
df[df.Credit_History == "Y"].Credit_History.value_counts()

,count
Credit_History,
0.0,0
1.0,0


In [119]:
df[df.Credit_History == "N"].Credit_History.value_counts()

,count
Credit_History,
0.0,0
1.0,0


In [120]:
df.columns

Index(['Dependents', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status'],
      dtype='object')

In [121]:
px.histogram(df, x="Property_Area", color="Loan_Status")

In [122]:
px.histogram(df[df.Property_Area=="Urban"], x="LoanAmount",  color="Loan_Status")

In [123]:
df[df.Credit_History == 0].Loan_Status.value_counts(True)

,proportion
Loan_Status,
N,0.921348
Y,0.078652


In [124]:
df[df.Credit_History == 1].Loan_Status.value_counts(True)

,proportion
Loan_Status,
Y,0.790476
N,0.209524


In [125]:
px.violin(df, x="Property_Area", y="LoanAmount", color="Loan_Status")

In [126]:
df.columns

Index(['Dependents', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status'],
      dtype='object')

In [127]:
px.histogram(df, x="CoapplicantIncome", color="Loan_Status")

In [128]:
df[df.CoapplicantIncome > 0].Loan_Status.value_counts(True)

,proportion
Loan_Status,
Y,0.718475
N,0.281525


In [129]:
df[df.CoapplicantIncome == 0].Loan_Status.value_counts(True)

,proportion
Loan_Status,
Y,0.648352
N,0.351648


In [130]:
df["isntalone"] =df.CoapplicantIncome == 0

In [131]:
df.head()

,Dependents,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,isntalone
0,0,No,5849,0.0,120.0,360.0,1.0,Urban,Y,True
1,1,No,4583,1508.0,128.0,360.0,1.0,Rural,N,False
2,0,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,True
3,0,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,False
4,0,No,6000,0.0,141.0,360.0,1.0,Urban,Y,True


# Split Input and Output

In [132]:
y = df[["Loan_Status"]]

In [133]:
df["ApplicantIncome"] = df.ApplicantIncome.astype("int")

In [134]:
df.CoapplicantIncome = df.CoapplicantIncome.astype("int")
df.LoanAmount =df.LoanAmount.astype("int")
df.Loan_Amount_Term = df.Loan_Amount_Term.astype("int")

In [135]:
X = df.drop("Loan_Status", axis=1)

In [136]:
X = pd.get_dummies(X, drop_first=True)

In [136]:
y = pd.get_dummies(y,drop_first=True)

# Decision Tree

# Evaluation

# K-Folds Cross Validation